In [24]:
import pandas as pd
import optuna
import numpy as np
import joblib

from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score, RandomizedSearchCV


In [9]:
df = pd.read_csv('df_att.csv')
target = 'Heart Disease'

X = df.drop(target, axis=1)
y = df[target]

cat_features = list(X.select_dtypes(exclude='number').columns)
num_features = list(X.select_dtypes(include='number').columns)


In [5]:
oof_mlp = np.load('oof_mlpC_preds.npy')
oof_logreg = np.load('oof_logreg.npy')
oof_cat_boost = np.load('oof_preds_catboost.npy')



In [13]:
X_meta = pd.DataFrame({'oof_mlp': oof_mlp, 'oof_logreg': oof_logreg, 'oof_cat_boost': oof_cat_boost})
y_meta = y.reset_index(drop=True).map({'Presence': 1, 'Absence': 0})

X_meta, y_meta

(         oof_mlp  oof_logreg  oof_cat_boost
 0       0.997057    0.997105       0.996944
 1       0.010934    0.017506       0.007891
 2       0.017618    0.018609       0.011107
 3       0.030615    0.056007       0.047372
 4       0.997773    0.998731       0.998368
 ...          ...         ...            ...
 629995  0.126048    0.122948       0.189705
 629996  0.494333    0.464025       0.509762
 629997  0.947244    0.949832       0.873899
 629998  0.440356    0.482126       0.496551
 629999  0.003378    0.004807       0.003388
 
 [630000 rows x 3 columns],
 0         1
 1         0
 2         0
 3         0
 4         1
          ..
 629995    0
 629996    0
 629997    1
 629998    1
 629999    0
 Name: Heart Disease, Length: 630000, dtype: int64)

xotebom


In [20]:
meta_model = LogisticRegression(
    max_iter = 2000, 
    verbose=0, 
    random_state=42,
    solver = 'lbfgs')


In [22]:
scores = cross_val_score(
    meta_model,
    X_meta,
    y_meta,
    cv=5,
    scoring='roc_auc',
    n_jobs=-1
)

print(scores.mean())


0.9554800784937093


In [26]:
param_dist = {
    'C': np.logspace(-3, 2, 50),
    'class_weight': [None, 'balanced']
}

search = RandomizedSearchCV(
    meta_model,
    param_distributions=param_dist,
    n_iter=20,
    scoring='roc_auc',
    cv=5,
    n_jobs=-1,
    random_state=42,
    verbose=1
)

search.fit(X_meta, y_meta)

print(search.best_score_)
print(search.best_params_)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
0.9554834800369392
{'class_weight': 'balanced', 'C': np.float64(15.264179671752334)}


In [28]:
search.get_params

<bound method BaseEstimator.get_params of RandomizedSearchCV(cv=5,
                   estimator=LogisticRegression(max_iter=2000, random_state=42),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'C': array([1.00000000e-03, 1.26485522e-03, 1.59985872e-03, 2.02358965e-03,
       2.55954792e-03, 3.23745754e-03, 4.09491506e-03, 5.17947468e-03,
       6.55128557e-03, 8.28642773e-03, 1.04811313e-02, 1.32571137e-02,
       1.67683294e-02, 2.12095089e-02, 2.68...
       1.84206997e+00, 2.32995181e+00, 2.94705170e+00, 3.72759372e+00,
       4.71486636e+00, 5.96362332e+00, 7.54312006e+00, 9.54095476e+00,
       1.20679264e+01, 1.52641797e+01, 1.93069773e+01, 2.44205309e+01,
       3.08884360e+01, 3.90693994e+01, 4.94171336e+01, 6.25055193e+01,
       7.90604321e+01, 1.00000000e+02]),
                                        'class_weight': [None, 'balanced']},
                   random_state=42, scoring='roc_auc', verbose=1)>

In [29]:
best_meta = search.best_estimator_
best_meta

,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",np.float64(15.264179671752334)
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",'balanced'
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",42
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide cla